In [ ]:
import os
import json
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [ ]:
assistant_id = "asst_5N02i9PqB3VVqUYSAbkm76WH"

assistant = client.beta.assistants.retrieve(assistant_id)
print(assistant)

In [ ]:
thread = client.beta.threads.create()
print(thread.id)

In [ ]:
# Add a user question to the thread
msg = ""
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=msg
)

thread_messages = client.beta.threads.messages.list(thread.id)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [ ]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

In [ ]:
from functions.documents import get_documents

import time

max_steps = 10 
sleep = .5

#add the available functions here   
available_functions = {"get_documents": get_documents}


try:
    cnt = 0
    while cnt < max_steps:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)        
        cnt += 1
        if run.status == "requires_action":
            tool_responses = []
            if (
                run.required_action.type == "submit_tool_outputs"
                and run.required_action.submit_tool_outputs.tool_calls is not None
            ):
                tool_calls = run.required_action.submit_tool_outputs.tool_calls

                for call in tool_calls:
                    if call.type == "function":
                        if call.function.name not in available_functions:
                            raise Exception("Function requested by the model does not exist")
                        function_to_call = available_functions[call.function.name]
                        print(call.function.arguments)
                        print(call.function.name)
                        tool_response = function_to_call(**json.loads(call.function.arguments))
                        tool_responses.append({"tool_call_id": call.id, "output": tool_response})

            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
            )

            print(run.status)

        if run.status == "failed":
            print("Run failed.")
            break
        if run.status == "completed":
            break
        time.sleep(sleep)

except Exception as e:
    print(e)



In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))
print(run.status)